In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os
import inspect
import getpass
import json

import pandas as pd
import matplotlib.pyplot as plt


project_path = os.path.abspath( os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))) + "/../" )
sys.path.insert(0, project_path)
print(project_path)

from database_storage.helper import *

from database_storage.database.database import *
from database_storage.main import *

from database_storage.objects.kpi import *

In [ ]:
def userinput(dict, node):
    
    if node in dict.keys():
        print(node)

        for properties in dict[node]: print("\t" + properties)
        property = input('Which property for the node? ' + node +'\n\t')

        operation = input('Which operation for the property "' + properties + '" ?\n\t')

        value = input('Which value for the property "' + properties + '" ?\n\t')

        return property , operation, value
    else:
        print(node, ' does not exists in your database')
        return None, None, None

# Database

In [ ]:
user = "neo4j"
password="neo4j"
bolt = "bolt://maintain.el.nist.gov:7687"

In [ ]:
databaseSchema_file = os.path.join(project_path ,'database_storage','database', 'DatabaseSchema.yaml')
databaseSchema_dict = openYAMLFile(databaseSchema_file)
database = DatabaseNeo4J(bolt, user, password, databaseSchema_dict)

In [ ]:
dict = database.getAllNode()

------

# Create the objects

## Issue

In [ ]:
prop, oper, val =  userinput(dict, "issue")
issue = OperandIssue(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(issue)

## Human

In [ ]:
prop, oper, val =  userinput(dict, "human")
human = OperandHuman(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(human)

## Technician

In [ ]:
prop, oper, val =  userinput(dict, "technician")
technician =OperandTechnician(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(technician)

## Operator

In [ ]:
prop, oper, val =  userinput(dict, "operator")
operator = OperandOperator(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(operator)

## Machine

In [ ]:
prop, oper, val =  userinput(dict, "machine")
machine = OperandMachine(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(machine)

## Machine Type

In [ ]:
prop, oper, val =  userinput(dict, "type of machine")
machinetype = OperandMachine(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(machinetype)

## Tags

In [ ]:
prop, oper, val =  userinput(dict, "all tags")
tag = OperandTag(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(tag)

## Tags - OneGram

In [ ]:
prop, oper, val =  userinput(dict, "tags one_gram")
one_gram = OperandTagOnegram(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(one_gram)

## Tags - NGram

In [ ]:
prop, oper, val =  userinput(dict, "tags n_gram")
n_gram = OperandTagNgram(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(n_gram)

## Tags - Other

In [ ]:
prop, oper, val =  userinput(dict, "tags other")
other = OperandTechnician(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(other)

## Tags - OneGram - Problem

In [ ]:
prop, oper, val =  userinput(dict, "tags problem")
problem = OperandTagProblem(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(problem)

## Tags - OneGram - Solution

In [ ]:
prop, oper, val =  userinput(dict, "tags solution")
solution = OperandTagSolution(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(solution)

## Tags - Other - NA

In [ ]:
prop, oper, val =  userinput(dict, "tags na")
na = OperandTagNa(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(na)

## Tags - Other - StopWord

In [ ]:
prop, oper, val =  userinput(dict, "tags stop_word")
stop_word = OperandTagStopWord(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(stop_word)

## Tags - NGram - ProblemItem

In [ ]:
prop, oper, val =  userinput(dict, "tags problem_item")
problem_item = OperandTagProblemItem(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(problem_item)

## Tags - NGram - SolutionItem

In [ ]:
prop, oper, val =  userinput(dict, "tags solution_item")
solution_item = OperandTagSolutionItem(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(solution_item)

## Tags - OneGram - Item

In [ ]:
prop, oper, val =  userinput(dict, "tags item")
item = OperandTagItem(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(item)

## Tags - OneGram - Item as problem

In [ ]:
prop, oper, val =  userinput(dict, "tag item as problem")
item_problem = OperandTagItemAsProblem(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(item_problem)

## Tags - OneGram - Item as solution

In [ ]:
prop, oper, val =  userinput(dict, "tag item as solution")
item_solution = OperandTagItemAsSolution(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(item_solution)

------

# RETURN

In [ ]:
### Tag Problem
res1 = OperandTagProblem(result = "keyword", databaseInfo = database.schema)
res2 = OperandTagItem(result = "keyword", databaseInfo = database.schema)

# KPI

In [ ]:
query = -technician - -machine + res1 + res2

In [ ]:
print(query)

In [ ]:
resultToObservationDataframe(database.runQuery(query.cypher_filterQuery())[1])

In [ ]:
print(query.cypher_filterQuery())

------

# KPI from "Developing meintenance key performance indocators from maintenance work order data"

In [ ]:
itemAsProblem = OperandTagItemAsProblem(result = "keyword", databaseInfo = database.schema)
machine = OperandMachine(result = "name", databaseInfo = database.schema)
machinetype = OperandMachine(result = "type", databaseInfo = database.schema)
technician = OperandTechnician(result = "name", databaseInfo = database.schema)
problemitem = OperandTagProblemItem(result = "keyword", databaseInfo = database.schema)
problem = OperandTagProblem(result = "keyword", databaseInfo = database.schema)

issue_dateMWOstart = OperandIssue(result = "date_maintenanceworkorder_start", databaseInfo = database.schema)
issue_dateMWOend = OperandIssue(result = "date_maintenanceworkorder_completion", databaseInfo = database.schema)

### 2.1.1 Common Problem Items

In [ ]:
kpi = itemAsProblem
df = resultToObservationDataframe(database.runQuery(kpi.cypher_filterQuery())[1])
df.head()

### 2.1.2 Common Problem Items per Time Between Failure

In [ ]:
kpi = itemAsProblem + issue_dateMWOstart + issue_dateMWOend
df = resultToObservationDataframe(database.runQuery(kpi.cypher_filterQuery())[1])
df.head()

### 2.1.3 Common Problem Items by Machine Type

In [ ]:
kpi = itemAsProblem + machinetype
df = resultToObservationDataframe(database.runQuery(kpi.cypher_filterQuery())[1])
df.head()

### 2.1.4 Common Problem Items & Action per Time to Repair

In [ ]:
kpi = problemitem + issue_dateMWOstart + issue_dateMWOend
df = resultToObservationDataframe(database.runQuery(kpi.cypher_filterQuery())[1])
df.head()

### 2.2.1 Machine per Time Between Failure

In [ ]:
kpi = machine + issue_dateMWOstart + issue_dateMWOend
df = resultToObservationDataframe(database.runQuery(kpi.cypher_filterQuery())[1])
df.head()

### 2.2.2 Machine by Problem Action per Time Between Failure

In [ ]:
kpi = machine + problem + issue_dateMWOstart + issue_dateMWOend
df = resultToObservationDataframe(database.runQuery(kpi.cypher_filterQuery())[1])
df.head()

### 2.2.3 Machine Type per Time to Repair

In [ ]:
kpi = machinetype + issue_dateMWOstart + issue_dateMWOend
df = resultToObservationDataframe(database.runQuery(kpi.cypher_filterQuery())[1])
df.head()

### 2.2.4 Machine type per Problem Item per Time to Repair

In [ ]:
kpi = machinetype +  itemAsProblem + issue_dateMWOstart + issue_dateMWOend
df = resultToObservationDataframe(database.runQuery(kpi.cypher_filterQuery())[1])
df.head()

### 2.3.1 Maintenance Technician Expertise per Problem Item

In [ ]:
kpi = technician + itemAsProblem
df = resultToObservationDataframe(database.runQuery(kpi.cypher_filterQuery())[1])
df.head()

### 2.3.2 Maintenance Technician Expertise per Problem Action

In [ ]:
kpi = technician + problem
df = resultToObservationDataframe(database.runQuery(kpi.cypher_filterQuery())[1])
df.head()

### 2.3.3 Maintenance Technician Expertise per Problem Item & Action per Time to Repair
### 2.3.4 Maintenance Technician Expertise per Problem Item & Action per Time to Diagnose
### 2.3.5 Maintenance Technician Expertise per Solution Item & Action per Time to Fix


In [ ]:
kpi = technician + problemitem + issue_dateMWOstart + issue_dateMWOend 
df = resultToObservationDataframe(database.runQuery(kpi.cypher_filterQuery())[1])
df.head()

### 2.3.6 Maintenance Technician Expertise per Machine Type per Time to Repair

In [ ]:
kpi = technician + machinetype + issue_dateMWOstart + issue_dateMWOend 
df = resultToObservationDataframe(database.runQuery(kpi.cypher_filterQuery())[1])
df.head()

---------

In [16]:
import os
from pathlib import Path

cwd = os.getcwd()

In [17]:
cwd

'/Users/sam11/Git/nestor/database_storage'

In [18]:
os.path.join(os.getcwd(), "3")

'/Users/sam11/Git/nestor/database_storage/3'